In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text
df_stock = pd.read_csv("itc/ITC.NS.csv",
                 parse_dates = ["Date"],
                 index_col = ["Date"])
df_posts = pd.read_excel("itc/itc_final_posts.xlsx")
# removing the unnecessary columns
df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
# removing spam posts
df_posts = df_posts[df_posts.Spam==0.0]
df_posts.drop(["Spam"],axis=1,inplace=True)
# sliding a window of 7 days and adding all the TIs
from stock_helper import prepare_data
x,y = prepare_data(df_stock)
# slicing the data
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]
# reversing the posts data
df_posts = df_posts[::-1]
final_posts = df_posts[7:]
# loading the sentiment analysis model
sent_model = tf.keras.models.load_model("final_bert")
# removing duplicates from the data
final_posts.drop_duplicates(subset=['Messages'])
# calculating the sentiments score
sentiments = []
prev = np.datetime64("2015-11-12 21:31:26")
for i in final_y.index:
    total=0
    cnt=0
    for j in final_posts.itertuples():
        _,msg,time = j
        if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
            total += tf.squeeze(sent_model.predict([msg])).numpy()
            cnt+=1
    prev = np.datetime64(i)
    if(cnt==0):
        sentiments.append(0)
    else:
        sentiments.append(total/cnt)
# getting indices where sentiments score is 0
zero_index = []
for i,j in enumerate(sentiments):
    if(j==0):
        zero_index.append(i)
# removing all the zero values indices
sentiments = np.delete(sentiments,zero_index)
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()
final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
# removing the added helper column
final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)

In [2]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
final_x_zeros_scaled = mms.fit_transform(final_x_zeros)
final_x_zeros_scaled

array([[0.61467075, 1.        , 0.9999995 , ..., 0.        , 0.14490831,
        0.        ],
       [0.8847647 , 0.91495323, 0.77971697, ..., 0.09165436, 0.28882992,
        0.        ],
       [0.9301157 , 0.89471245, 0.7361207 , ..., 0.2254045 , 0.39610314,
        0.        ],
       ...,
       [0.24333572, 0.32355213, 0.42327833, ..., 0.6687889 , 0.52184355,
        0.26923072],
       [0.23974895, 0.3533659 , 0.45800447, ..., 0.8273011 , 0.85380137,
        0.6153846 ],
       [0.23554707, 0.38004112, 0.48953772, ..., 0.984114  , 0.9999999 ,
        0.65384614]], dtype=float32)

In [3]:
final_y_zeros


,Close
Date,
2021-11-15,238.100006
2021-11-16,235.449997
2021-11-17,239.500000
2021-11-18,237.500000
2021-12-03,221.649994
2021-12-06,217.899994
2021-12-07,220.649994
2021-12-08,224.949997
2021-12-09,235.399994


In [4]:
k = final_y_zeros.max()
final_y_zeros= final_y_zeros/k
final_y_zeros

,Close
Date,
2021-11-15,0.994155
2021-11-16,0.983090
2021-11-17,1.000000
2021-11-18,0.991649
2021-12-03,0.925470
2021-12-06,0.909812
2021-12-07,0.921294
2021-12-08,0.939248
2021-12-09,0.982881


In [5]:
model_nbeats = tf.keras.models.load_model("nbeats_ITC")
model_nbeats.evaluate(final_x_zeros[42:],final_y_zeros[42:]*k)

1/1 [==============================] - 1s 679ms/step - loss: 2.3721 - mae: 2.3721 - mse: 7.8516


[2.3720510005950928, 2.3720510005950928, 7.851637363433838]

In [16]:
tf.random.set_seed(42)
inputs = tf.keras.layers.Input(shape=(71))
x = tf.keras.layers.Lambda(lambda x : tf.expand_dims(x,axis=1))(inputs)
x = tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.LSTM(128)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs = inputs, outputs = output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 71)]              0         
                                                                 
 lambda_1 (Lambda)           (None, 1, 71)             0         
                                                                 
 lstm_3 (LSTM)               (None, 1, 128)            102400    
                                                                 
 lstm_4 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_5 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_6 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_7 (LSTM)               (None, 128)               1315

In [17]:
model.compile(loss="mae",optimizer = tf.keras.optimizers.Adam(), metrics=["mae","mse"])
history = model.fit(final_x_zeros_scaled[:42],final_y_zeros[:42],
                    epochs = 1000,
                    validation_data = (final_x_zeros_scaled[42:], final_y_zeros[42:]),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 1/1000
2/2 [==============================] - 6s 1s/step - loss: 0.9268 - mae: 0.9268 - mse: 0.8598 - val_loss: 0.9393 - val_mae: 0.9393 - val_mse: 0.8830 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 58ms/step - loss: 0.9158 - mae: 0.9158 - mse: 0.8397 - val_loss: 0.9270 - val_mae: 0.9270 - val_mse: 0.8600 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 59ms/step - loss: 0.9031 - mae: 0.9031 - mse: 0.8166 - val_loss: 0.9118 - val_mae: 0.9118 - val_mse: 0.8320 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 51ms/step - loss: 0.8874 - mae: 0.8874 - mse: 0.7885 - val_loss: 0.8918 - val_mae: 0.8918 - val_mse: 0.7959 - lr: 0.0010
Epoch 5/1000
2/2 [==============================] - 0s 48ms/step - loss: 0.8661 - mae: 0.8661 - mse: 0.7511 - val_loss: 0.8634 - val_mae: 0.8634 - val_mse: 0.7461 - lr: 0.0010
Epoch 6/1000
2/2 [==============================] - 0s 51ms/step - loss: 0.8362 - mae: 0.8362 - mse: 0.7001 - val_loss: 0.

Epoch 93/1000
2/2 [==============================] - 0s 44ms/step - loss: 0.0237 - mae: 0.0237 - mse: 7.9605e-04 - val_loss: 0.0186 - val_mae: 0.0186 - val_mse: 6.1708e-04 - lr: 0.0010
Epoch 94/1000
2/2 [==============================] - 0s 37ms/step - loss: 0.0222 - mae: 0.0222 - mse: 7.8877e-04 - val_loss: 0.0470 - val_mae: 0.0470 - val_mse: 0.0025 - lr: 0.0010
Epoch 95/1000
2/2 [==============================] - 0s 36ms/step - loss: 0.0218 - mae: 0.0218 - mse: 6.8544e-04 - val_loss: 0.0499 - val_mae: 0.0499 - val_mse: 0.0028 - lr: 0.0010
Epoch 96/1000
2/2 [==============================] - 0s 38ms/step - loss: 0.0246 - mae: 0.0246 - mse: 9.8845e-04 - val_loss: 0.0196 - val_mae: 0.0196 - val_mse: 6.7759e-04 - lr: 0.0010
Epoch 97/1000
2/2 [==============================] - 0s 44ms/step - loss: 0.0251 - mae: 0.0251 - mse: 9.9291e-04 - val_loss: 0.0180 - val_mae: 0.0180 - val_mse: 5.7277e-04 - lr: 0.0010
Epoch 98/1000
2/2 [==============================] - 0s 36ms/step - loss: 0.0224 - 

Epoch 138/1000
2/2 [==============================] - 0s 35ms/step - loss: 0.0215 - mae: 0.0215 - mse: 6.5966e-04 - val_loss: 0.0312 - val_mae: 0.0312 - val_mse: 0.0011 - lr: 0.0010
Epoch 139/1000
2/2 [==============================] - 0s 35ms/step - loss: 0.0194 - mae: 0.0194 - mse: 6.1201e-04 - val_loss: 0.0511 - val_mae: 0.0511 - val_mse: 0.0027 - lr: 0.0010
Epoch 140/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0247 - mae: 0.0247 - mse: 8.9726e-04 - val_loss: 0.0178 - val_mae: 0.0178 - val_mse: 4.0502e-04 - lr: 0.0010
Epoch 141/1000
2/2 [==============================] - 0s 38ms/step - loss: 0.0200 - mae: 0.0200 - mse: 5.4297e-04 - val_loss: 0.0085 - val_mae: 0.0085 - val_mse: 1.0491e-04 - lr: 0.0010
Epoch 142/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0216 - mae: 0.0216 - mse: 7.0129e-04 - val_loss: 0.0275 - val_mae: 0.0275 - val_mse: 8.3600e-04 - lr: 0.0010
Epoch 143/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0

2/2 [==============================] - 0s 33ms/step - loss: 0.0137 - mae: 0.0137 - mse: 2.6349e-04 - val_loss: 0.0252 - val_mae: 0.0252 - val_mse: 6.7195e-04 - lr: 0.0010
Epoch 227/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0141 - mae: 0.0141 - mse: 2.9619e-04 - val_loss: 0.0251 - val_mae: 0.0251 - val_mse: 6.6732e-04 - lr: 0.0010
Epoch 228/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0154 - mae: 0.0154 - mse: 3.6868e-04 - val_loss: 0.0245 - val_mae: 0.0245 - val_mse: 6.3902e-04 - lr: 0.0010
Epoch 229/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0148 - mae: 0.0148 - mse: 3.2262e-04 - val_loss: 0.0321 - val_mae: 0.0321 - val_mse: 0.0011 - lr: 0.0010
Epoch 230/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0139 - mae: 0.0139 - mse: 3.0752e-04 - val_loss: 0.0206 - val_mae: 0.0206 - val_mse: 4.6006e-04 - lr: 0.0010
Epoch 231/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0133 - mae: 

2/2 [==============================] - 0s 34ms/step - loss: 0.0112 - mae: 0.0112 - mse: 1.9555e-04 - val_loss: 0.0277 - val_mae: 0.0277 - val_mse: 8.2582e-04 - lr: 1.0000e-04
Epoch 314/1000
2/2 [==============================] - 0s 35ms/step - loss: 0.0110 - mae: 0.0110 - mse: 1.9826e-04 - val_loss: 0.0241 - val_mae: 0.0241 - val_mse: 6.3828e-04 - lr: 1.0000e-04
Epoch 315/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0092 - mae: 0.0092 - mse: 1.3477e-04 - val_loss: 0.0211 - val_mae: 0.0211 - val_mse: 5.0220e-04 - lr: 1.0000e-04
Epoch 316/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0119 - mae: 0.0119 - mse: 2.2305e-04 - val_loss: 0.0198 - val_mae: 0.0198 - val_mse: 4.4664e-04 - lr: 1.0000e-04
Epoch 317/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0132 - mae: 0.0132 - mse: 2.6678e-04 - val_loss: 0.0201 - val_mae: 0.0201 - val_mse: 4.5897e-04 - lr: 1.0000e-04
Epoch 318/1000
2/2 [==============================] - 0s 35ms/ste

2/2 [==============================] - 0s 37ms/step - loss: 0.0125 - mae: 0.0125 - mse: 2.4402e-04 - val_loss: 0.0297 - val_mae: 0.0297 - val_mse: 9.3517e-04 - lr: 1.0000e-04
Epoch 400/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0105 - mae: 0.0105 - mse: 1.8351e-04 - val_loss: 0.0298 - val_mae: 0.0298 - val_mse: 9.3919e-04 - lr: 1.0000e-04
Epoch 401/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0119 - mae: 0.0119 - mse: 2.2511e-04 - val_loss: 0.0299 - val_mae: 0.0299 - val_mse: 9.4839e-04 - lr: 1.0000e-04
Epoch 402/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0106 - mae: 0.0106 - mse: 1.7843e-04 - val_loss: 0.0283 - val_mae: 0.0283 - val_mse: 8.5462e-04 - lr: 1.0000e-04
Epoch 403/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0126 - mae: 0.0126 - mse: 2.2430e-04 - val_loss: 0.0265 - val_mae: 0.0265 - val_mse: 7.5487e-04 - lr: 1.0000e-04
Epoch 404/1000
2/2 [==============================] - 0s 33ms/ste

In [13]:
model.compile(loss="mae",optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["mae","mse"])
history = model.fit(final_x_zeros_scaled[:43],final_y_zeros[:43],
                    epochs = 2000,
                    initial_epoch = history.epoch[-1],
                    validation_data = (final_x_zeros_scaled[43:], final_y_zeros[43:]),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 481/2000
2/2 [==============================] - 3s 689ms/step - loss: 0.0145 - mae: 0.0145 - mse: 2.8724e-04 - val_loss: 0.0241 - val_mae: 0.0241 - val_mse: 7.3832e-04 - lr: 1.0000e-04
Epoch 482/2000
2/2 [==============================] - 0s 39ms/step - loss: 0.0088 - mae: 0.0088 - mse: 1.1659e-04 - val_loss: 0.0315 - val_mae: 0.0315 - val_mse: 0.0011 - lr: 1.0000e-04
Epoch 483/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.0084 - mae: 0.0084 - mse: 1.2071e-04 - val_loss: 0.0283 - val_mae: 0.0283 - val_mse: 9.5502e-04 - lr: 1.0000e-04
Epoch 484/2000
2/2 [==============================] - 0s 42ms/step - loss: 0.0099 - mae: 0.0099 - mse: 1.8099e-04 - val_loss: 0.0197 - val_mae: 0.0197 - val_mse: 5.4131e-04 - lr: 1.0000e-04
Epoch 485/2000
2/2 [==============================] - 0s 42ms/step - loss: 0.0085 - mae: 0.0085 - mse: 1.0012e-04 - val_loss: 0.0152 - val_mae: 0.0152 - val_mse: 3.8071e-04 - lr: 1.0000e-04
Epoch 486/2000
2/2 [==============================] -

2/2 [==============================] - 0s 33ms/step - loss: 0.0079 - mae: 0.0079 - mse: 1.0393e-04 - val_loss: 0.0223 - val_mae: 0.0223 - val_mse: 7.0171e-04 - lr: 1.0000e-04
Epoch 568/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.0064 - mae: 0.0064 - mse: 6.5647e-05 - val_loss: 0.0203 - val_mae: 0.0203 - val_mse: 6.1380e-04 - lr: 1.0000e-04
Epoch 569/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0084 - mae: 0.0084 - mse: 1.1608e-04 - val_loss: 0.0218 - val_mae: 0.0218 - val_mse: 6.7748e-04 - lr: 1.0000e-04
Epoch 570/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0070 - mae: 0.0070 - mse: 6.7865e-05 - val_loss: 0.0259 - val_mae: 0.0259 - val_mse: 8.7421e-04 - lr: 1.0000e-04
Epoch 571/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0079 - mae: 0.0079 - mse: 9.9966e-05 - val_loss: 0.0294 - val_mae: 0.0294 - val_mse: 0.0011 - lr: 1.0000e-04
Epoch 572/2000
2/2 [==============================] - 0s 33ms/step - 

2/2 [==============================] - 0s 33ms/step - loss: 0.0077 - mae: 0.0077 - mse: 8.8096e-05 - val_loss: 0.0256 - val_mae: 0.0256 - val_mse: 8.5721e-04 - lr: 1.0000e-05
Epoch 653/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0077 - mae: 0.0077 - mse: 8.9094e-05 - val_loss: 0.0262 - val_mae: 0.0262 - val_mse: 8.8973e-04 - lr: 1.0000e-05
Epoch 654/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0059 - mae: 0.0059 - mse: 6.7445e-05 - val_loss: 0.0267 - val_mae: 0.0267 - val_mse: 9.1699e-04 - lr: 1.0000e-05
Epoch 655/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0099 - mae: 0.0099 - mse: 1.6056e-04 - val_loss: 0.0271 - val_mae: 0.0271 - val_mse: 9.3813e-04 - lr: 1.0000e-05
Epoch 656/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0092 - mae: 0.0092 - mse: 1.4327e-04 - val_loss: 0.0270 - val_mae: 0.0270 - val_mse: 9.3533e-04 - lr: 1.0000e-05
Epoch 657/2000
2/2 [==============================] - 0s 33ms/ste

In [18]:
preds = tf.squeeze(model.predict(final_x_zeros_scaled[42:])).numpy()
preds = preds*k.values[0]
preds

array([214.65805, 216.33624, 219.05553, 225.66968, 230.60774, 233.77744,
       232.622  , 231.33575, 230.19392, 229.92274, 230.17053],
      dtype=float32)

In [19]:
mae = sum(abs(preds-(tf.squeeze(final_y_zeros[42:].to_numpy()).numpy())*k.values[0]))
mae

13.919815063476562

In [20]:
model.save("LSTM_itc")

INFO:tensorflow:Assets written to: LSTM_itc\assets


INFO:tensorflow:Assets written to: LSTM_itc\assets


In [21]:
f = tf.keras.models.load_model("LSTM_itc")
preds = tf.squeeze(f.predict(final_x_zeros_scaled[42:])).numpy()
preds = preds*k.values[0]

In [22]:
mae = sum(abs(preds-(tf.squeeze(final_y_zeros[42:].to_numpy()).numpy())*k.values[0]))
mae

13.919815063476562